# RoBERTa Base Model Evaluation on IMDB Dataset

This notebook evaluates the performance of the un-fine-tuned RoBERTa base model on the IMDB movie review dataset using zero-shot classification.

## Overview
- **Model**: RoBERTa-base (un-fine-tuned)
- **Dataset**: IMDB Movie Reviews (50,000 reviews)
- **Task**: Sentiment Classification (Positive/Negative)
- **Method**: Zero-shot classification

## Dependencies
Install required packages:
```bash
pip install kagglehub[pandas-datasets] transformers torch scikit-learn tqdm pandas numpy
```


In [1]:
!pip install kagglehub[pandas-datasets] transformers torch scikit-learn tqdm pandas numpy


In [2]:
# Import required libraries
import kagglehub
import pandas as pd
import numpy as np
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
from tqdm import tqdm
import torch
import os


## Configuration

Set the configuration parameters for the evaluation:


In [3]:
# Configuration
MODEL_NAME = "roberta-base"
SAMPLE_SIZE = None  # Set to None to use full dataset, or a number (e.g., 1000) for testing
BATCH_SIZE = 600
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")


Using device: cuda


## Initialize RoBERTa Model

Load the RoBERTa base model and initialize the zero-shot classifier:


In [4]:
print(f"Loading RoBERTa base model: {MODEL_NAME}")

# Initialize zero-shot classifier
classifier = pipeline(
    "zero-shot-classification",
    model=MODEL_NAME,
    device=0 if DEVICE == "cuda" else -1
)

print("Model loaded successfully!")


Loading RoBERTa base model: roberta-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting

Model loaded successfully!


## Load IMDB Dataset

Download and load the IMDB movie review dataset from Kaggle:


In [5]:
# Load IMDB dataset from Kaggle
print("Loading IMDB dataset from Kaggle...")
dataset_handle = "lakshmi25npathi/imdb-dataset-of-50k-movie-reviews"
csv_filename = "IMDB Dataset.csv"

# Download dataset and load CSV file directly with proper encoding
dataset_path = kagglehub.dataset_download(dataset_handle)
csv_file_path = os.path.join(dataset_path, csv_filename)

# Try different encodings to handle the file properly
encodings = ['latin-1', 'iso-8859-1', 'cp1252', 'utf-8']
df = None

for encoding in encodings:
    try:
        df = pd.read_csv(csv_file_path, encoding=encoding)
        print(f"Successfully loaded {csv_filename} with {encoding} encoding")
        break
    except (UnicodeDecodeError, LookupError) as e:
        continue

if df is None:
    raise ValueError(f"Could not load CSV file with any of the tried encodings: {encodings}")

print(f"\nDataset loaded: {len(df)} records")
print(f"Dataset columns: {df.columns.tolist()}")
print("\nFirst 5 records:")
print(df.head())


Loading IMDB dataset from Kaggle...
Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.
Successfully loaded IMDB Dataset.csv with latin-1 encoding

Dataset loaded: 50000 records
Dataset columns: ['review', 'sentiment']

First 5 records:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


## Explore Dataset

Check dataset structure and sentiment distribution:


In [6]:
# Check dataset structure
print("Dataset info:")
print(df.info())
print(f"\nSentiment distribution:")
print(df['sentiment'].value_counts())


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None

Sentiment distribution:
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


## Prepare Data

Map sentiment labels to numerical values and optionally sample the dataset:


In [7]:
# Prepare data
# Map sentiment to labels for evaluation
sentiment_mapping = {'positive': 1, 'negative': 0}
df['label'] = df['sentiment'].str.lower().map(sentiment_mapping)

# Sample data if specified
if SAMPLE_SIZE:
    print(f"\nSampling {SAMPLE_SIZE} records for evaluation...")
    df = df.sample(n=min(SAMPLE_SIZE, len(df)), random_state=42)
    df = df.reset_index(drop=True)
    print(f"Using {len(df)} samples for evaluation")
else:
    print(f"Using full dataset: {len(df)} samples")


Using full dataset: 50000 samples


## Zero-Shot Classification

Perform zero-shot classification on the IMDB reviews:


In [8]:
# Zero-shot classification labels
candidate_labels = ["positive", "negative"]

print(f"\nEvaluating on {len(df)} reviews...")
print("This may take a while...")

# Perform zero-shot classification
predictions = []
scores = []

# Process reviews in batches
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing reviews"):
    batch_texts = df['review'].iloc[i:i+BATCH_SIZE].tolist()

    # Zero-shot classifier processes batches automatically
    try:
        batch_results = classifier(batch_texts, candidate_labels)
        # Results might be a list or single dict depending on batch size
        if not isinstance(batch_results, list):
            batch_results = [batch_results]

        for result in batch_results:
            # Get the predicted label (highest score)
            predicted_label = result['labels'][0]
            predicted_score = result['scores'][0]
            predictions.append(1 if predicted_label == "positive" else 0)
            scores.append(predicted_score)
    except Exception as e:
        # If batch processing fails, process one by one
        print(f"Batch processing failed, processing individually: {e}")
        for text in batch_texts:
            result = classifier(text, candidate_labels)
            predicted_label = result['labels'][0]
            predicted_score = result['scores'][0]
            predictions.append(1 if predicted_label == "positive" else 0)
            scores.append(predicted_score)

print(f"\nCompleted processing {len(predictions)} reviews")



Evaluating on 50000 reviews...
This may take a while...


Processing reviews: 100%|██████████| 84/84 [33:57<00:00, 24.25s/it]


Completed processing 50000 reviews


## Calculate Metrics

Compute evaluation metrics:


In [9]:
# Convert to numpy arrays
y_true = df['label'].values
y_pred = np.array(predictions)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, average='weighted', zero_division=0
)
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred, average='macro', zero_division=0
)


## Evaluation Results

Display comprehensive evaluation results:


In [10]:
# Print results
print("="*60)
print("ROBERTA BASE MODEL EVALUATION RESULTS")
print("="*60)
print(f"\nModel: {MODEL_NAME}")
print(f"Dataset: IMDB Movie Reviews")
print(f"Number of samples: {len(df)}")
print(f"Evaluation method: Zero-shot classification")

print(f"\n{'Metric':<20} {'Value':<15}")
print("-" * 35)
print(f"{'Accuracy':<20} {accuracy:.4f}")
print(f"{'Precision (weighted)':<20} {precision:.4f}")
print(f"{'Recall (weighted)':<20} {recall:.4f}")
print(f"{'F1-score (weighted)':<20} {f1:.4f}")
print(f"{'Precision (macro)':<20} {precision_macro:.4f}")
print(f"{'Recall (macro)':<20} {recall_macro:.4f}")
print(f"{'F1-score (macro)':<20} {f1_macro:.4f}")


ROBERTA BASE MODEL EVALUATION RESULTS

Model: roberta-base
Dataset: IMDB Movie Reviews
Number of samples: 50000
Evaluation method: Zero-shot classification

Metric               Value          
-----------------------------------
Accuracy             0.5614
Precision (weighted) 0.5617
Recall (weighted)    0.5614
F1-score (weighted)  0.5609
Precision (macro)    0.5617
Recall (macro)       0.5614
F1-score (macro)     0.5609


In [11]:
print("\nDetailed Classification Report:")
print(classification_report(y_true, y_pred,
                          target_names=['negative', 'positive'],
                          digits=4))



Detailed Classification Report:
              precision    recall  f1-score   support

    negative     0.5574    0.5963    0.5762     25000
    positive     0.5660    0.5266    0.5456     25000

    accuracy                         0.5614     50000
   macro avg     0.5617    0.5614    0.5609     50000
weighted avg     0.5617    0.5614    0.5609     50000



In [12]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("\n(rows = true labels, columns = predicted labels)")
print("     [negative  positive]")
print("\n" + "="*60)
print("Evaluation complete!")
print("="*60)



Confusion Matrix:
[[14907 10093]
 [11835 13165]]

(rows = true labels, columns = predicted labels)
     [negative  positive]

Evaluation complete!
